# Prototype-based Machine Learning on Distance Data

In this small demo, we show how prototype-based machine learning on distance data works.
Assume, for example, that you have a data set of strings, which you would like to cluster.

In [1]:
X = ['ctctagag', 'cctcggag', 'tcctagga', 'tcttagaa', 'ggagcctc', 'gagactct', 'agagtctc', 'aagattct']

Such a data set does not have a straightforward vectorial representation. However, it is pretty easy to describe it in terms of pairwise _distances_, namely the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance).

In [2]:
import numpy as np

# define the levenshtein distance
def levenshtein(x, y):
    m = len(x)
    n = len(y)
    Delta = np.zeros((m+1, n+1), dtype=int)
    for i in range(m):
        Delta[i+1, 0] = Delta[i, 0] + 1
    for j in range(n):
        Delta[0, j+1] = Delta[0, j] + 1
    for i in range(m):
        for j in range(n):
            delta_ij = 0 if x[i] == y[j] else 1
            Delta[i+1, j+1] = np.min([delta_ij + Delta[i,j], 1 + Delta[i+1, j], 1 + Delta[i, j+1]])
    return Delta[m][n]

# compute the pairwise Levenshtein distance between all strings
D = np.zeros((len(X), len(X)))
for k in range(len(X)):
    x = X[k]
    for l in range(k+1, len(X)):
        y = X[l]
        # compute the Levenshtein distance between x and y
        # and store it symmetrically
        D[k, l] = levenshtein(x, y)
        D[l, k] = D[k, l]

Now that we have computed a distance matrix D, we can cluster this data easily using _relational neural gas_ as follows.
We only need to pre-define the number of clusters $K$ we expect, which are two in this case.

In [3]:
from proto_dist_ml.rng import RNG
rng = RNG(K = 2)
rng = rng.fit(D)

Let's see which clusters the model predicts on our data:

In [4]:
print(rng.predict(D))

[0 0 0 0 1 1 1 1]


In other words, the first four strings belong to one cluster, the latter four strings to the other cluster.

We can even make cluster predictions for new data, if we compute the distances to all training data.

In [5]:
X_new = ['ccctgagg', 'aggatcct']

D_new = np.zeros((len(X_new), len(X)))
for k in range(len(X_new)):
    x = X_new[k]
    for l in range(len(X)):
        y = X[l]
        D_new[k, l] = levenshtein(x, y)

print(rng.predict(D_new))

[0 1]


In other words, the first new string belongs to the first cluster, the second new string to the second.

## Classification

In our example case, the pattern in the data is quite easy to spot: The first cluster consists of strings which have 'c' and 't' letters in front and 'a' and 'g' letters in the back, while the other cluster consists of strings where this pattern is inverted. Now that we spotted this rule, we can _assign_ class labels to our strings and train a classifier instead of a clustering model. In this case, we use a RGLVQ classifier.

In [6]:
# set up the 'true' class labels
y = np.array([0, 0, 0, 0, 1, 1, 1, 1])

# set up an RGLVQ model with one prototype per class
from proto_dist_ml.rglvq import RGLVQ
rglvq = RGLVQ(K = 1)
# train it
rglvq = rglvq.fit(D, y)
# check the classification accuracy
print('Our RGLVQ classifier has an accuracy of %g%% on the training data' % (rglvq.score(D, y) * 100))

Our RGLVQ classifier has an accuracy of 100% on the training data


Let's check how our model generalizes to our new data.

In [7]:
# set up 'true' class labels
y_new = np.array([0, 1])
print('Our RGLVQ classifier has an accuracy of %g%% on the test data' % (rglvq.score(D_new, y_new) * 100))

Our RGLVQ classifier has an accuracy of 100% on the test data


### Median Classification

A remaining drawback of RGLVQ is that we still need to compute the distance of new data to _all_ our training data, which can become expensive in application. Further, it is difficult to interpret our prototypes just based on their convex coefficients. A remedy for both issues is offered by _median_ classification, where each prototype is set to exactly one data point. We can train a median GLVQ model as follows.

In [8]:
# set up a MGLVQ model with one prototype per class
from proto_dist_ml.mglvq import MGLVQ
mglvq = MGLVQ(K = 1)
# train it
mglvq = mglvq.fit(D, y)
# check the classification accuracy
print('Our MGLVQ classifier has an accuracy of %g%% on the training data' % (mglvq.score(D, y) * 100))

Our MGLVQ classifier has an accuracy of 100% on the training data


Let's have a look at our prototypes:

In [9]:
print('Prototype for class 0: %s' % X[mglvq._w[0]])
print('Prototype for class 1: %s' % X[mglvq._w[1]])

Prototype for class 0: tcctagga
Prototype for class 1: gagactct


To classify new data, we only need to compute the distance to our prototypes now, which are just two points instead of eight.

In [10]:
D_new_small = D_new[:, mglvq._w]
print('Our MGLVQ classifier has an accuracy of %g%% on the test data' % (mglvq.score(D_new_small, y_new) * 100))

Our MGLVQ classifier has an accuracy of 100% on the test data
